In [1]:
%matplotlib inline

In [2]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from tqdm.auto import tqdm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [4]:
from transformers import WhisperConfig, WhisperModel
from transformers import AutoConfig

configuration = AutoConfig.from_pretrained('./whisper/config.json')
model = WhisperModel(configuration)
configuration = model.config

In [5]:
checkpoint = torch.load('./whisper/whisper_small_ru.bin')

In [6]:
n = {}
for key in checkpoint:
    n_key = key.replace("model.", "")
    n[n_key] = checkpoint[key]

In [7]:
model.load_state_dict(n)

<All keys matched successfully>

In [8]:
from transformers import AutoFeatureExtractor, WhisperTokenizer, WhisperProcessor

feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer(vocab_file='./whisper/vocab.json', 
                             merges_file='./whisper/merges.txt',
                             normalizer_file='./whisper/normalizer.json')
processor = WhisperProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor

WhisperProcessor:
- feature_extractor: WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: WhisperTokenizer(name_or_path='', vocab_size=50258, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftranscript|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [9]:
waveform, sample_rate = torchaudio.load('./records_by_classes/good/2022-12-16_16-58-43.wav', normalize = True, channels_first = True)
waveform.shape

torch.Size([1, 240000])

In [10]:
waveform.squeeze().numpy()

array([ 0.00131226,  0.00140381,  0.00085449, ..., -0.00091553,
       -0.00167847, -0.0012207 ], dtype=float32)

In [11]:
from datasets import load_dataset

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")


Found cached dataset librispeech_asr_dummy (C:/Users/ioci/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [12]:
sample = next(iter(ds))
sample

{'file': 'C:/Users/ioci/.cache/huggingface/datasets/downloads/extracted/63064962313fba1167095c45e0b093546e9ea4e3fa8f3659ed940deea0a628df/dev_clean/1272/128104\\1272-128104-0000.flac',
 'audio': {'path': 'C:/Users/ioci/.cache/huggingface/datasets/downloads/extracted/63064962313fba1167095c45e0b093546e9ea4e3fa8f3659ed940deea0a628df/dev_clean/1272/128104\\1272-128104-0000.flac',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
         0.0010376 ]),
  'sampling_rate': 16000},
 'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL',
 'speaker_id': 1272,
 'chapter_id': 128104,
 'id': '1272-128104-0000'}

In [13]:
a = ds[0]["audio"]["array"]
a

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ])

In [14]:
a.shape

(93680,)

In [12]:
inputs = processor(ds[0]["audio"]["array"], sampling_rate = ds[0]["audio"]["sampling_rate"], return_tensors="pt")
input_features = inputs.input_features

In [ ]:
input_features = inputs.input_features
generated_ids = model.generate(input_features=input_features)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
transcription

TypeError: conv1d() received an invalid combination of arguments - got (str, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple of (int,)!, !tuple of (int,)!, !tuple of (int,)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple of (int,)!, !tuple of (int,)!, !tuple of (int,)!, int)
